**Notes**

+ Models are trained to determine semantically similar texts, what is not always the case for songs.

In [1]:
%pip install  torch
%pip install  transformers
%pip install  accelerate
%pip install  sentence-transformers
%pip install  langchain
%pip install  langchain-community
%pip install  sentence-transformers
%pip install  tranfromers
%pip install  datasets

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
ERROR: Could not find a version that satisfies the requirement tranfromers (from versions: none)
ERROR: No matching distribution found for tranfromers
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import pandas as pd
from langchain.embeddings import HuggingFaceEmbeddings
from datasets import load_dataset,Dataset, concatenate_datasets
import numpy as np
from time import process_time
import math
import random
import joblib

from sentence_transformers import SentenceTransformer,SentenceTransformerTrainingArguments
from sentence_transformers.losses import BatchAllTripletLoss, BatchHardSoftMarginTripletLoss
from sentence_transformers.trainer import SentenceTransformerTrainer
from sentence_transformers.training_args import BatchSamplers
from sentence_transformers.evaluation import TripletEvaluator

import torch

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

/home/kaba4ok/Documents/oles_territory/studies_in_Lviv/year3/sem1/ai/project/gesser/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


**Embedder part**

In [3]:
model = SentenceTransformer("sentence-transformers/all-distilroberta-v1")

In [4]:
dataset = load_dataset("csv", data_files="./tcc_ceds_music.csv")
dataset = dataset.remove_columns(["artist_name", "track_name"])
dataset = dataset.rename_columns({"genre":"label", "lyrics":"sentence"})

In [5]:
unique_genres = dataset['train'].unique('label')
genre_to_index = {label: idx for idx, label in enumerate(unique_genres)}

def label_to_index_func(data_set):
    data_set['label'] = genre_to_index[data_set['label']]
    return data_set

dataset = dataset.map(label_to_index_func)

In [6]:
# populate with permuted lyrics 
n_songs_to_permute = len(dataset['train'])
n_permutations = 4

permuted_lyrics = []
lables_of_permuted = []

for i in range(n_songs_to_permute):
    song_item = dataset['train'][i]
    label = song_item['label']
    sentence = song_item['sentence'].split(' ')

    for ii in range(n_permutations):
        permuted_song = " ".join(random.sample(sentence, len(sentence)))
        permuted_lyrics.append(permuted_song)
        lables_of_permuted.append(label)

permuted_dataset = Dataset.from_dict({"sentence":permuted_lyrics, "label":lables_of_permuted})
dataset['train'] = concatenate_datasets([dataset['train'], permuted_dataset])

In [7]:
train_test_split = dataset['train'].train_test_split(test_size=0.05)

In [8]:
loss = BatchHardSoftMarginTripletLoss(model)
training_args = SentenceTransformerTrainingArguments(
    # Required parameter:
    output_dir="./models/guesser_2",
    # Optional training parameters:
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    learning_rate=2e-5,
    warmup_ratio=0.15,
    adam_beta1=0.85,
    adam_beta2=0.99,
    weight_decay=0.01,
    use_cpu=False,
    fp16=True,  # Set to False if you get an error that your GPU can't run on FP16
    bf16=False,  # Set to True if you have a GPU that supports BF16
    batch_sampler=BatchSamplers.NO_DUPLICATES,  # MultipleNegativesRankingLoss benefits from no duplicate samples in a batch
    # Optional tracking/debugging parameters:
    save_steps=100,
    save_total_limit=2,
    logging_steps=50,
    seed=135)

    # eval_strategy="steps",
    # eval_steps=100,
    # save_strategy="steps",

: 

In [ ]:
train_set = train_test_split['train']
trainer = SentenceTransformerTrainer(
    model=model,
    train_dataset=train_set,
    loss=loss,
    args=training_args
)

trainer.train()

  0%|          | 25/25269 [01:35<23:49:56,  3.40s/it]

In [ ]:
model.save_pretrained("trained-embedder")

In [ ]:
! tar -czf trained_embedder.tar.gz ./trained-embedder

tar: ./trained-embedder: Cannot stat: No such file or directory
tar: Exiting with failure status due to previous errors


In [ ]:
#creates test dataset to evaluate the embedder

from collections import defaultdict
import random

class_groups = defaultdict(list)
for example in train_test_split['test']:
    class_groups[example['label']].append(example['sentence'])

triplets = []

for label, sentences in class_groups.items():
    for i in range(len(sentences)):
        for j in range(i + 1, len(sentences)):
            anchor = sentences[i]
            positive = sentences[j]

            negative_label = random.choice([lbl for lbl in class_groups.keys() if lbl != label])
            negative = random.choice(class_groups[negative_label])

            triplets.append({
                'anchor': anchor,
                'positive': positive,
                'negative': negative
            })

triplet_test_dataset = Dataset.from_list(triplets)
triplet_test_dataset = triplet_test_dataset.shuffle(seed=83)

print(triplet_test_dataset)

Dataset({
    features: ['anchor', 'positive', 'negative'],
    num_rows: 175370
})


In [ ]:
test_evaluator = TripletEvaluator(
    anchors=triplet_test_dataset["anchor"],
    positives=triplet_test_dataset["positive"],
    negatives=triplet_test_dataset["negative"],
    name="all-test",
)
print(test_evaluator(model))

{'all-test_cosine_accuracy': 0.5246108228317272, 'all-test_dot_accuracy': 0.4753891771682728, 'all-test_manhattan_accuracy': 0.5225067001197469, 'all-test_euclidean_accuracy': 0.5246108228317272, 'all-test_max_accuracy': 0.5246108228317272}


In [ ]:
! tar -xzf ./trained_embedder.tar.gz ./

In [6]:
model = SentenceTransformer("trained-embedder")

In [7]:
def get_lyrics_embedding(lyrics:str):
  return model.encode([lyrics.lower()])

**Estimator part**

In [8]:
#creates a dataframe of all songs as embeddings -> genre as idx

lyrics_embeddigs = []
genres = []
unique_genres = set()

import string
songs_and_lyrics = pd.read_csv('./tcc_ceds_music.csv')

for idx in range(len(songs_and_lyrics)):
  genre = songs_and_lyrics["genre"][idx]
  unique_genres.add(genre)

  lyrics_as_bag_of_words_string = str(songs_and_lyrics["lyrics"][idx])

  embedding =  get_lyrics_embedding(lyrics_as_bag_of_words_string)[0]
  lyrics_embeddigs.append(embedding)
  genres.append(genre)

embedding_set = pd.DataFrame({"genres":genres})
embeddings_list = []

embedding_size = model.get_sentence_embedding_dimension()

for i in range(embedding_size):
  dim_i = pd.Series([embedding[i] for embedding in lyrics_embeddigs])
  embeddings_list.append(dim_i)

embeddings_df = pd.concat(embeddings_list, axis=1)
embedding_set = pd.concat([embeddings_df, embedding_set], axis=1)

embedding_set['genres'] = embedding_set['genres'].apply(lambda genre: genre_to_index[genre])

In [9]:
joblib.dump(embedding_set, "embedding_set_of_all_songs.pkl")

['embedding_set_of_all_songs.pkl']

In [67]:
#transforms the training embedding data into torch-compliant format

features = embedding_set.iloc[:, :-1].values
labels = embedding_set.iloc[:, -1].values
sparse_labels = []
for genre_label in labels:
  sparse_arr = [0 for i in range(len(unique_genres))]
  sparse_arr[genre_label] = 1
  sparse_labels.append(sparse_arr)

features_tensor = torch.tensor(features, dtype=torch.float32)
labels_tensor = torch.tensor(sparse_labels, dtype=torch.float32)

dataset_train = torch.utils.data.TensorDataset(features_tensor, labels_tensor)

train_data_loader = torch.utils.data.DataLoader(dataset_train, batch_size=5, shuffle=True)

#transforms the testing embedding data into torch-compliant format

test_size = 8000
rand_indices = np.random.randint(0, len(embedding_set), size=test_size)

features = embedding_set.iloc[rand_indices, :-1].values
labels = embedding_set.iloc[rand_indices, -1].values

features_tensor = torch.tensor(features, dtype=torch.float32)
labels_tensor = torch.tensor(labels, dtype=torch.int32)

dataset_test = torch.utils.data.TensorDataset(features_tensor, labels_tensor)

test_data_loader = torch.utils.data.DataLoader(dataset_test, batch_size=5, shuffle=True)

In [85]:
#the nn used for genre estimation

class NeuralGuesser(nn.Module):
    def __init__(self, embedding_size:int):
        super().__init__()
        self.pool = nn.AvgPool1d(kernel_size=2, padding=0) #gives me embedding_size/3 inputs

        self.layer1 = nn.Linear(int(embedding_size/2), int(embedding_size/3))
        self.layer2 = nn.Linear(int(embedding_size/3), int(embedding_size/6))
        self.layer3 = nn.Linear(int(embedding_size/6), int(embedding_size/12))

        self.layer4 = nn.Linear(int(embedding_size/12), len(unique_genres))

    def forward(self, x):
        x = F.relu(self.pool(x))
        x = F.relu(self.layer1(x))
        x = F.relu(self.layer2(x))
        x = F.relu(self.layer3(x))
        x = self.layer4(x)

        return x

In [99]:
#training the nn

guesser = NeuralGuesser(embedding_size)
loss_alg = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(guesser.parameters(), lr=0.001, betas=(0.9, 0.99))

for epoch in range(3):

    running_loss = 0.0

    for i, data in enumerate(train_data_loader, 0):
        inputs, labels = data

        optimizer.zero_grad()
        outputs = guesser(inputs)
        loss = loss_alg(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 500 == 499:
            print(f'[{epoch + 1}, {i + 1:5d}] average loss: {running_loss / 500:.3f}')
            running_loss = 0.0

torch.save(guesser.state_dict(), "./trained-nn.pth")

[1,   500] average loss: 1.845
[1,  1000] average loss: 1.811
[1,  1500] average loss: 1.781
[1,  2000] average loss: 1.739
[1,  2500] average loss: 1.737
[1,  3000] average loss: 1.742
[1,  3500] average loss: 1.721
[1,  4000] average loss: 1.738
[1,  4500] average loss: 1.701
[1,  5000] average loss: 1.707
[1,  5500] average loss: 1.733
[2,   500] average loss: 1.712
[2,  1000] average loss: 1.678
[2,  1500] average loss: 1.708
[2,  2000] average loss: 1.693
[2,  2500] average loss: 1.722
[2,  3000] average loss: 1.684
[2,  3500] average loss: 1.700
[2,  4000] average loss: 1.681
[2,  4500] average loss: 1.673
[2,  5000] average loss: 1.676
[2,  5500] average loss: 1.676
[3,   500] average loss: 1.667
[3,  1000] average loss: 1.667
[3,  1500] average loss: 1.671
[3,  2000] average loss: 1.655
[3,  2500] average loss: 1.642
[3,  3000] average loss: 1.655
[3,  3500] average loss: 1.659
[3,  4000] average loss: 1.675
[3,  4500] average loss: 1.655
[3,  5000] average loss: 1.655
[3,  550

In [100]:
correct = 0
total = 0
with torch.no_grad():
    for data in test_data_loader:
        images, labels = data
        outputs = guesser(images)

        value, predicted_classes = torch.max(outputs.data, 1)

        total += labels.size(0)
        correct += (predicted_classes == labels).sum().item()

print(f'Accuracy for 8000 test images: {100 * correct // total} %')

Accuracy for 8000 test images: 33 %


In [36]:
%pip install nltk
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('punkt_tab')

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
4534.74s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


Note: you may need to restart the kernel to use updated packages.


[nltk_data] Downloading package stopwords to
[nltk_data]     /home/kaba4ok/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/kaba4ok/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /home/kaba4ok/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [37]:
stop_words = set(stopwords.words('english'))
symbols = set([",",".","/","#","*","(",")","?","+","=",":",";","%","[", "]"])
song_specific = set(["chorus", "repeat", "verse", "oh", "ooh", "ah", "aah"])

def check_if_not_to_be_removed(token:str):
  return (not token.lower() in stop_words) and (not token.lower() in symbols) and (not token.lower() in song_specific) and (not any(char.isdigit() for char in token)) and (not "'" in token)

def clean_lyrics(lyrics:str):
  tokenized_lyrics = word_tokenize(lyrics)
  # tokenized_lyrics = lyrics.split()
  filtered_lyrics = [w for w in tokenized_lyrics if check_if_not_to_be_removed(w)]

  lyrics_concat=""
  for token in filtered_lyrics:
    lyrics_concat+=f" {token} "

  return lyrics_concat

In [101]:
def estimate_genre(model, song_lyrics):
  filtered_lyrics = clean_lyrics(song_lyrics)

  print(f"Filtered lyrics: {filtered_lyrics}")

  embedding = get_lyrics_embedding(filtered_lyrics)
  print(embedding)
  prediction = model(torch.tensor(embedding, dtype=torch.float32))

  value, genre = torch.max(prediction, 1)

  for key,item in genre_to_index.items():
    if genre == item:
      return key

  return -1


In [102]:
lyrics="Ooh You can dance You can jive Having the time of your life Ooh, see that girl Watch that scene Digging the dancing queen Friday night and the lights are low Looking out for a place to go Where they play the right music Getting in the swing You come to look for a king Anybody could be that guy Night is young and the music's high With a bit of rock music Everything is fine You're in the mood for a dance And when you get the chance You are the dancing queen Young and sweet Only seventeen Dancing queen Feel the beat from the tambourine, oh yeah You can dance You can jive Having the time of your life Ooh, see that girl Watch that scene Digging the dancing queen You're a teaser, you turn 'em on Leave 'em burning and then you're gone Looking out for another Anyone will do You're in the mood for a dance And when you get the chance You are the dancing queen Young and sweet Only seventeen Dancing queen Feel the beat from the tambourine, oh yeah You can dance You can jive Having the time of your life Ooh, see that girl Watch that scene Digging the dancing queen Digging the dancing queen"
print(estimate_genre(guesser, lyrics))

Filtered lyrics:  dance  jive  time  life  see  girl  Watch  scene  Digging  dancing  queen  Friday  night  lights  low  Looking  place  go  play  right  music  Getting  swing  come  look  king  Anybody  could  guy  Night  young  music  high  bit  rock  music  Everything  fine  mood  dance  get  chance  dancing  queen  Young  sweet  seventeen  Dancing  queen  Feel  beat  tambourine  yeah  dance  jive  time  life  see  girl  Watch  scene  Digging  dancing  queen  teaser  turn  Leave  burning  gone  Looking  another  Anyone  mood  dance  get  chance  dancing  queen  Young  sweet  seventeen  Dancing  queen  Feel  beat  tambourine  yeah  dance  jive  time  life  see  girl  Watch  scene  Digging  dancing  queen  Digging  dancing  queen 
[[-2.22006328e-02 -9.39148664e-02  3.59825827e-02 -8.37531406e-04
  -3.28827985e-02 -1.59182400e-02  6.45272713e-03  1.61068104e-02
   4.72730957e-02  1.38637302e-02 -4.93043773e-02 -1.01293493e-02
  -3.43856849e-02  2.04595700e-02  4.55711596e-02 -3.6285385

In [96]:
lyrics="Half past twelve And I'm watchin' the late show in my flat, all alone How I hate to spend the evening on my own Autumn winds Blowin' outside the window as I look around the room And it makes me so depressed to see the gloom There's not a soul out there No one to hear my prayer Gimme, gimme, gimme a man after midnight Won't somebody help me chase these shadows away? Gimme, gimme, gimme a man after midnight Take me through the darkness to the break of the day Movie stars Find the end of the rainbow with a fortune to win It's so different from the world I'm living in Tired of TV I open the window and I gaze into the night But there's nothing there to see, no one in sight There's not a soul out there No one to hear my prayer Gimme, gimme, gimme a man after midnight Won't somebody help me chase these shadows away? Gimme, gimme, gimme a man after midnight Take me through the darkness to the break of the day Gimme, gimme, gimme a man after midnight Gimme, gimme, gimme a man after midnight There's not a soul out there No one to hear my prayer Gimme, gimme, gimme a man after midnight Won't somebody help me chase these shadows away? Gimme, gimme, gimme a man after midnight Take me through the darkness to the break of the day Gimme, gimme, gimme a man after midnight Won't somebody help me chase these shadows away? Gimme, gimme, gimme a man after midnight Take me through the darkness to the break of the day"
print(estimate_genre(guesser, lyrics))

Filtered lyrics:  Half  past  twelve  watchin  late  show  flat  alone  hate  spend  evening  Autumn  winds  Blowin  outside  window  look  around  room  makes  depressed  see  gloom  soul  one  hear  prayer  Gim  gim  gim  man  midnight  Wo  somebody  help  chase  shadows  away  Gim  gim  gim  man  midnight  Take  darkness  break  day  Movie  stars  Find  end  rainbow  fortune  win  different  world  living  Tired  TV  open  window  gaze  night  nothing  see  one  sight  soul  one  hear  prayer  Gim  gim  gim  man  midnight  Wo  somebody  help  chase  shadows  away  Gim  gim  gim  man  midnight  Take  darkness  break  day  Gim  gim  gim  man  midnight  Gim  gim  gim  man  midnight  soul  one  hear  prayer  Gim  gim  gim  man  midnight  Wo  somebody  help  chase  shadows  away  Gim  gim  gim  man  midnight  Take  darkness  break  day  Gim  gim  gim  man  midnight  Wo  somebody  help  chase  shadows  away  Gim  gim  gim  man  midnight  Take  darkness  break  day 
reggae


In [97]:
lyrics="You'll take my life but I'll take yours, too You'll fire your musket but I'll run you through So when you're waiting for the next attack You'd better stand, there's no turning back The bugle sounds, the charge begins But on this battlefield, no one wins The smell of acrid smoke and horses' breath As I plunge on into certain death Oh Oh The horse he sweats with fear, we break to run The mighty roar of the Russian guns And as we race towards the human wall The screams of pain as my comrades fall We hurdle bodies that lay on the ground And the Russians fire another round We get so near yet so far away We won't live to fight another day Oh Oh We get so close, near enough to fight When a Russian gets me in his sights He pulls the trigger and I feel the blow A burst of rounds take my horse below And as I lay there gazing at the sky My body's numb and my throat is dry And as I lay forgotten and alone Without a tear I draw my parting groan Oh Oh"
print(estimate_genre(guesser, lyrics))

Filtered lyrics:  take  life  take  fire  musket  run  waiting  next  attack  better  stand  turning  back  bugle  sounds  charge  begins  battlefield  one  wins  smell  acrid  smoke  horses  breath  plunge  certain  death  horse  sweats  fear  break  run  mighty  roar  Russian  guns  race  towards  human  wall  screams  pain  comrades  fall  hurdle  bodies  lay  ground  Russians  fire  another  round  get  near  yet  far  away  wo  live  fight  another  day  get  close  near  enough  fight  Russian  gets  sights  pulls  trigger  feel  blow  burst  rounds  take  horse  lay  gazing  sky  body  numb  throat  dry  lay  forgotten  alone  Without  tear  draw  parting  groan 
pop


In [98]:
lyrics="How I've waited for you to come I've been here all alone Now that you've arrived Please stay awhile And I promise I won't keep you long I'll keep you forever (forever, forever, forever) Graze the skin with my fingertips The brush of dead, cold flesh appease the means Provoking images, delicate features so smooth A pleasant fragrance in the light of the moon Dance with the dead in my dreams Listen to their hallowed screams The dead have taken my soul Temptation's lost all control Simple smiles elude psychotic eyes Lose all mind control, rationale declines Empty eyes enslave the creations Of placid faces and lifeless pageants In the depths of a mind insane Fantasy and reality are the same Graze the skin with my fingertips The brush of dead, warm flesh pacifies the means Incised members ornaments on my being Adulating the skin before me Simple smiles elude psychotic eyes Lose all mind control, rationale declines Empty eyes enslave the creations Of placid faces and lifeless pageants Dance with the dead in my dreams (Hello? Hello, Mr. Gein?) Listen to their hallowed screams (Mr. Gein?) The dead have taken my soul (let me out of here, Mr. Gein) Temptation's lost all control (Mr. Gein? I don't want to play anymore, Mr. Gein) Dance with the dead in my dreams (Mr. Gein, this isn't fun anymore) Listen to their hallowed screams (I don't wanna play anymore, Mr. Gein, Mr. Gein?) The dead have taken my soul (I want out of here, Mr. Gein) Temptation's lost all control (let me out now)"
print(estimate_genre(guesser, lyrics))

Filtered lyrics:  waited  come  alone  arrived  Please  stay  awhile  promise  wo  keep  long  keep  forever  forever  forever  forever  Graze  skin  fingertips  brush  dead  cold  flesh  appease  means  Provoking  images  delicate  features  smooth  pleasant  fragrance  light  moon  Dance  dead  dreams  Listen  hallowed  screams  dead  taken  soul  Temptation  lost  control  Simple  smiles  elude  psychotic  eyes  Lose  mind  control  rationale  declines  Empty  eyes  enslave  creations  placid  faces  lifeless  pageants  depths  mind  insane  Fantasy  reality  Graze  skin  fingertips  brush  dead  warm  flesh  pacifies  means  Incised  members  ornaments  Adulating  skin  Simple  smiles  elude  psychotic  eyes  Lose  mind  control  rationale  declines  Empty  eyes  enslave  creations  placid  faces  lifeless  pageants  Dance  dead  dreams  Hello  Hello  Mr  Gein  Listen  hallowed  screams  Mr  Gein  dead  taken  soul  let  Mr.  Gein  Temptation  lost  control  Mr.  Gein  want  play  